In [4]:
import math
import csv
import json

STAT_NAMES = [
    "caps", "garbage_time_caps", "hold", "ndps", "returns", "quick_returns", "nrts", "pups"
]

with open("smurfs.json") as f:
    smurfs = json.load(f)

def desmurf(player):
    player = player.replace("\\'", "'").replace("\\\\", "\\")
    if player in smurfs:
        return smurfs[player]
    return player

def row_to_dict(row):
    return {
        'match_id': int(row[0]),
        'timestamp': int(row[1]),
        'duration': int(row[2]),
        'cap_diff': int(row[3]),
        'garbage_time_cap_diff': int(row[4]),
        'players': [
            {
                'name': desmurf(row[j + 5]),
                'stats': {
                    stat_name: int(row[j * len(STAT_NAMES) + 13 + i])
                    for i, stat_name in enumerate(STAT_NAMES)
                }
            }
            for j in range(8)
        ]
    }

with open("matchups_with_stats.csv") as f:
    reader = csv.reader(f)
    next(reader)
    matches = [row_to_dict(row) for row in reader]

elo = {}
variance = {}
pred_errors = []
winner_win_prob = []

In [5]:
STARTING_VARIANCE = 1.2
VARIANCE_TO_ADD_BACK = 0.002
BASE_VARIANCE = 4.6
RED_ADVANTAGE = 0.1
PLAYER_STAT_WEIGHTS_INITIAL = {
    "caps": 0.8,
    "garbage_time_caps": -0.3,
    "hold": 1.8 / 3600,
    "ndps": -0.5,
    "returns": 0.35,
    "quick_returns": 0.0,
    "nrts": 0.25,
    "pups": 0.3
}
TEAM_STAT_WEIGHTS_INITIAL = {
    "caps": 0,
    "garbage_time_caps": -0.3,
    "hold": 0.8 / 3600,
    "ndps": -0.0,
    "returns": 0.04,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0
}
PLAYER_STAT_WEIGHTS_CONVERGED = {
    "caps": 0.05,
    "garbage_time_caps": -0.0,
    "hold": 0.0 / 3600,
    "ndps": -0.0,
    "returns": 0.0,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0
}
TEAM_STAT_WEIGHTS_CONVERGED = {
    "caps": 0,
    "garbage_time_caps": -0.0,
    "hold": 0.45 / 3600,
    "ndps": -0.0,
    "returns": 0.02,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0
}
DIFF_MAPPING = [0, 0.3, 1.0, 2.1, 3.1, 4.2, -4.2, -3.1, -2.1, -1.0, -0.3]
NEW_PLAYER_ELO = -1.05
NEWNESS_THRESHOLD = 0.05
NEW_PLAYER_ELO_BOOST = 0.1
DIFF_WEIGHT_INITIAL = 0.6
DIFF_WEIGHT_CONVERGED = 0.75
GARBAGE_TIME_DISCOUNT = 0.4

In [6]:
def get_elo(player):
    if player not in elo:
        elo[player] = NEW_PLAYER_ELO
    return elo[player]

def get_variance(player):
    if player not in variance:
        variance[player] = STARTING_VARIANCE
    return variance[player]

def normalize_elos():
    known_elos = [elo[p] for p in elo if variance[p] <= 0.2]
    avg_elo = sum(known_elos) / max(10, len(known_elos))
    for p in elo:
        elo[p] -= avg_elo
        variance[p] = min(STARTING_VARIANCE, variance[p] + VARIANCE_TO_ADD_BACK)

In [7]:
def judge_stats_initial(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def judge_stats_converged(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def update_rating(player, error, stat_score_initial, stat_score_converged, total_variance):
    share_of_variance = variance[player] / total_variance
    newness = variance[player] / STARTING_VARIANCE
    error_weight = DIFF_WEIGHT_INITIAL * newness + DIFF_WEIGHT_CONVERGED * (1 - newness)
    stat_score = stat_score_initial * newness + stat_score_converged * (1 - newness)
    update = error_weight * error + stat_score
    elo[player] += update * share_of_variance
    variance[player] *= 1 - share_of_variance
    if variance[player] > NEWNESS_THRESHOLD:
        elo[player] += NEW_PLAYER_ELO_BOOST * share_of_variance

def update_ratings(match):
    expected_score = RED_ADVANTAGE
    total_variance = BASE_VARIANCE
    for player in match['players'][:4]:
        expected_score += get_elo(player['name'])
        total_variance += get_variance(player['name'])
    for player in match['players'][4:]:
        expected_score -= get_elo(player['name'])
        total_variance += get_variance(player['name'])
    winner_win_prob.append(1 / (1 + math.exp(-expected_score / 2 if match['cap_diff'] > 0 else expected_score / 2)))
    pred_errors.append(match['cap_diff'] - min(max(expected_score, -5), 5))
    error = DIFF_MAPPING[match['cap_diff']] -\
        match['garbage_time_cap_diff'] * GARBAGE_TIME_DISCOUNT -\
        min(max(expected_score, DIFF_MAPPING[-5]), DIFF_MAPPING[5])
    red_stats_initial, blue_stats_initial = judge_stats_initial(match['players'])
    red_stats_converged, blue_stats_converged = judge_stats_converged(match['players'])
    for player, stat_score_initial, stat_score_converged in zip(match['players'][:4], red_stats_initial, red_stats_converged):
        update_rating(player['name'], error, stat_score_initial, stat_score_converged, total_variance)
    for player, stat_score_initial, stat_score_converged in zip(match['players'][4:], blue_stats_initial, blue_stats_converged):
        update_rating(player['name'], -error, stat_score_initial, stat_score_converged, total_variance)

In [8]:
elo = {}
variance = {}
pred_errors = []
winner_win_prob = []
day = 0

for match in matches:
    new_day = round(match['timestamp'] / 86400)
    if 1 in elo:
        print("what?")
    if new_day != day:
        normalize_elos()
    day = new_day
    update_ratings(match)

print(f"MAE: {sum([abs(e) for e in pred_errors]) / len(pred_errors):.4f}    MSE: {sum([e ** 2 for e in pred_errors]) / len(pred_errors):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob]) / len(winner_win_prob):.2%}    WWP: {sum(winner_win_prob) / len(winner_win_prob):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob]) / len(winner_win_prob)) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob]) / len(winner_win_prob)):.2%}")

MAE: 2.3876    MSE: 8.3130
COR: 59.82%    WWP: 52.79%
BRI: 51.57%    LOG: 51.64%


In [9]:
print(f"MAE: {sum([abs(e) for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}    MSE: {sum([e ** 2 for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:]):.2%}    WWP: {sum(winner_win_prob[10000:]) / len(winner_win_prob[10000:]):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])):.2%}")

MAE: 2.3813    MSE: 8.2654
COR: 58.79%    WWP: 52.25%
BRI: 51.24%    LOG: 51.27%


In [10]:
leaderboard = sorted([
    p for p in elo if variance[p] <= 0.3
], key=lambda x: elo[x], reverse=True)
for i, player in enumerate(leaderboard):
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}")

  1.  okthen         3.40 ± 0.45
  2.  SluffAndRuff   2.99 ± 0.60
  3.  DT             2.94 ± 0.49
  4.  OuchMyBalls    2.91 ± 0.37
  5.  CarrotCake     2.90 ± 0.34
  6.  phreak         2.75 ± 0.43
  7.  toasty         2.73 ± 0.77
  8.  Alphachurro    2.67 ± 0.43
  9.  Ritual         2.54 ± 0.45
 10.  jig            2.48 ± 0.48
 11.  fender         2.41 ± 0.44
 12.  meowza         2.38 ± 0.34
 13.  mex            2.34 ± 0.37
 14.  Xx360NoSwagx   2.29 ± 0.85
 15.  Shikari        2.29 ± 0.49
 16.  BALLDON'TLIE   2.28 ± 0.41
 17.  danp           2.24 ± 0.41
 18.  Suchit         2.21 ± 0.38
 19.  eee            2.13 ± 0.54
 20.  Enervate       2.12 ± 0.35
 21.  realtea        2.11 ± 0.41
 22.  bitter heart   2.05 ± 1.05
 23.  ft             2.03 ± 0.45
 24.  Crippy         2.02 ± 0.57
 25.  grunt          2.00 ± 0.88
 26.  Vader          2.00 ± 0.42
 27.  Messi          1.99 ± 0.47
 28.  ASAP           1.99 ± 0.51
 29.  Madoka         1.95 ± 0.84
 30.  Ball-E         1.92 ± 0.76
 31.  Ty  

In [11]:
import random

ranks = []
for _ in range(10000):
    simulated_elos = {p: elo[p] + variance[p] ** 0.5 * random.gauss(0, 1) for p in elo if variance[p] <= 0.3}
    sorted_players = sorted(simulated_elos.items(), key=lambda x: x[1], reverse=True)
    ranks.append({player: rank + 1 for rank, (player, _) in enumerate(sorted_players)})

for i, player in enumerate(leaderboard[:]):
    rank_ci = sorted([r[player] for r in ranks])[500:-500]
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}  ({min(rank_ci)}-{max(rank_ci)})")

  1.  okthen         3.40 ± 0.45  (1-3)
  2.  SluffAndRuff   2.99 ± 0.60  (1-12)
  3.  DT             2.94 ± 0.49  (1-11)
  4.  OuchMyBalls    2.91 ± 0.37  (2-10)
  5.  CarrotCake     2.90 ± 0.34  (2-9)
  6.  phreak         2.75 ± 0.43  (3-15)
  7.  toasty         2.73 ± 0.77  (1-25)
  8.  Alphachurro    2.67 ± 0.43  (3-17)
  9.  Ritual         2.54 ± 0.45  (5-23)
 10.  jig            2.48 ± 0.48  (5-26)
 11.  fender         2.41 ± 0.44  (7-28)
 12.  meowza         2.38 ± 0.34  (8-26)
 13.  mex            2.34 ± 0.37  (8-29)
 14.  Xx360NoSwagx   2.29 ± 0.85  (4-56)
 15.  Shikari        2.29 ± 0.49  (8-38)
 16.  BALLDON'TLIE   2.28 ± 0.41  (9-34)
 17.  danp           2.24 ± 0.41  (10-36)
 18.  Suchit         2.21 ± 0.38  (11-36)
 19.  eee            2.13 ± 0.54  (10-49)
 20.  Enervate       2.12 ± 0.35  (14-40)
 21.  realtea        2.11 ± 0.41  (13-44)
 22.  bitter heart   2.05 ± 1.05  (5-96)
 23.  ft             2.03 ± 0.45  (14-51)
 24.  Crippy         2.02 ± 0.57  (11-59)
 25.  grunt